In [1]:
%matplotlib inline

import itertools
import re

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np


def map_array(map_txt):
    rows = map_txt.splitlines()
    map_ = np.array([list(r) for r in rows])
    return map_


def print_map(map_):
    for row in map_:
        print(''.join(row))

        
def out_of_bounds(pos, n, m):
    return (pos[0] < 0) | (pos[0] >= n) | (pos[1] < 0) | (pos[1] >= m)


directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

In [2]:
txt = """
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
""".strip()

In [3]:
with open('input.txt', 'r') as f:
    txt = f.read().strip()

# Part 1

In [4]:
operand_map = {
    0: lambda: 0,
    1: lambda: 1,
    2: lambda: 2,
    3: lambda: 3,
    4: lambda: registers['A'],
    5: lambda: registers['B'],
    6: lambda: registers['C'],
    7: lambda: 0/0,
}


def adv(operand, registers, pointer):
    operand_value = operand_map[operand]()
    #registers['A'] = int(registers['A'] / 2 ** operand_value)
    registers['A'] = registers['A'] >> operand_value
    return pointer + 2, []


def bxl(operand, registers, pointer):
    registers['B'] = registers['B'] ^ operand
    return pointer + 2, []
    

def bst(operand, registers, pointer):
    operand_value = operand_map[operand]()
    # registers['B'] = operand_value % 8
    registers['B'] = operand_value & 7
    return pointer + 2, []
    
    
def jnz(operand, registers, pointer):
    if registers['A'] == 0:
        return pointer + 2, []
    else:
        return operand, []


def bxc(operand, registers, pointer):
    registers['B'] = registers['B'] ^ registers['C']
    return pointer + 2, []


def out(operand, registers, pointer):
    #operand_value = operand_map[operand]() % 8
    operand_value = operand_map[operand]() & 7
    return pointer + 2, [operand_value]


def bdv(operand, registers, pointer):
    operand_value = operand_map[operand]()
    registers['B'] = registers['A'] >> operand_value
    return pointer + 2, []


def cdv(operand, registers, pointer):
    operand_value = operand_map[operand]()
    registers['C'] = registers['A'] >> operand_value
    return pointer + 2, []

    
opcode_map = {
    0: adv,
    1: bxl,
    2: bst,
    3: jnz,
    4: bxc,
    5: out,
    6: bdv,
    7: cdv,
}

In [6]:
reg, prog = txt.split('\n\n')
registers = {}
for r, line in zip(['A', 'B', 'C'], reg.splitlines()):
    registers[r] = int(line[11:])
    
program = [int(x) for x in prog[8:].split(',')]

In [7]:
registers, program

({'A': 33024962, 'B': 0, 'C': 0},
 [2, 4, 1, 3, 7, 5, 1, 5, 0, 3, 4, 2, 5, 5, 3, 0])

In [8]:
def execute(program, registers):
    pointer = 0
    outputs = []
    while len(program) > pointer:
        opcode = program[pointer]
        operand = program[pointer + 1]
        pointer, out = opcode_map[opcode](operand, registers, pointer)
        outputs.extend(out)
    return outputs

registers = {
    'A': 0,
    'B': 0,
    'C': 0,
}

reg, prog = txt.split('\n\n')
for r, line in zip(['A', 'B', 'C'], reg.splitlines()):
    registers[r] = int(line[11:])
program = [int(x) for x in prog[8:].split(',')]

outputs = execute(program, registers)
','.join(str(x) for x in outputs)



'5,1,3,4,3,7,2,1,7'

In [9]:
outputs

[5, 1, 3, 4, 3, 7, 2, 1, 7]

# Part 2

In [10]:
txt = """
Register A: 117440
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0
""".strip()

In [11]:
with open('input.txt', 'r') as f:
    txt = f.read().strip()

In [12]:
reg, prog = txt.split('\n\n')
program = [int(x) for x in prog[8:].split(',')]


In [13]:
def execute_step(a):
    b = (a & 7) ^ 3
    c = a >> b
    b = (b ^ 5) ^ c
    return b & 7


good = {0: {0}}
for level in range(16):
    target = program[-level-1]
    good[level+1] = set()
    for candidate in good[level]:

        a_start = candidate << 3
        a_stop = a_start + 8
        for a in range(a_start, a_stop):
            out = execute_step(a)
            if out == target:
                good[level+1].add(a)
    print(good)



{0: {0}, 1: {6}}
{0: {0}, 1: {6}, 2: {49, 53}}
{0: {0}, 1: {6}, 2: {49, 53}, 3: {393, 425, 397, 430, 429}}
{0: {0}, 1: {6}, 2: {49, 53}, 3: {393, 425, 397, 430, 429}, 4: {3401, 3145, 3177, 3433, 3407, 3151, 3442}}
{0: {0}, 1: {6}, 2: {49, 53}, 3: {393, 425, 397, 430, 429}, 4: {3401, 3145, 3177, 3433, 3407, 3151, 3442}, 5: {25213, 25165, 25166, 27213, 27214, 25422, 27470, 27261}}
{0: {0}, 1: {6}, 2: {49, 53}, 3: {393, 425, 397, 430, 429}, 4: {3401, 3145, 3177, 3433, 3407, 3151, 3442}, 5: {25213, 25165, 25166, 27213, 27214, 25422, 27470, 27261}, 6: {218093, 201709}}
{0: {0}, 1: {6}, 2: {49, 53}, 3: {393, 425, 397, 430, 429}, 4: {3401, 3145, 3177, 3433, 3407, 3151, 3442}, 5: {25213, 25165, 25166, 27213, 27214, 25422, 27470, 27261}, 6: {218093, 201709}, 7: {1744744, 1613678, 1613672, 1744750}}
{0: {0}, 1: {6}, 2: {49, 53}, 3: {393, 425, 397, 430, 429}, 4: {3401, 3145, 3177, 3433, 3407, 3151, 3442}, 5: {25213, 25165, 25166, 27213, 27214, 25422, 27470, 27261}, 6: {218093, 201709}, 7: {174474

In [14]:
min(good[16])

216584205979245